In [2]:
import pandas as pd
import numpy as np
import pickle
from konlpy.tag import Kkma
from tqdm import tqdm
from gensim.models import FastText as ft
from __future__ import print_function
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import re
from nltk.tokenize import RegexpTokenizer
import nltk
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import pad_sequence
from konlpy.tag import Hannanum
from tqdm import tqdm
from konlpy.tag import Okt
from gensim.models.phrases import Phrases

In [3]:
with open('seoul_oneroom_pr.pkl','rb') as seoulroom:
    df = pickle.load(seoulroom)

In [4]:
df

,id,lat,lng,info,url,address,detail,picture,type,deposit,rental_fee,maintenance_fee,doc
144,34402581,37.446504,127.057650,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 서초구 신원동,"◆◆전입가,무융자,전세대출가,지하철역3분◆이중창,주차가능 ◆◆전입가,무융자,전세대출...",https://ic.zigbang.com/ic/items/34402581/1.jpg,전세,17500,NaN,8.0,서울시 서초구 신원동 전입가 무융자 전세대출가 지하철역 분 이중창 주차가능 전입가 ...
145,34448303,37.446532,127.057644,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 서초구 신원동,청계산입구 초 역세권 오피스텔 *청계산입구 초 역세권 신축 오피스텔\r\n*강남역3...,https://ic.zigbang.com/ic/items/34448303/1.jpg,월세,1000,65.0,10.0,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 청계산입구 초 역세권 신축 오피...
159,34521223,37.447032,127.056570,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/345...,서울시 서초구 신원동,★신축급 풀옵션 투룸 오피스텔 전세 매물★ # 12월8일 이후부터 투어 가능\r\...,https://ic.zigbang.com/ic/items/34521223/1.jpg,전세,37500,NaN,20.0,서울시 서초구 신원동 신축급 풀옵션 투룸 오피스텔 전세 매물 월 일 이후부터 투어 ...
165,34493219,37.446730,127.056288,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 서초구 신원동,청계산입구 초 역세권 오피스텔 *청계산입구 초 역세권 신축 오피스텔\r\n*강남역3...,https://ic.zigbang.com/ic/items/34493219/1.jpg,전세,17000,NaN,8.0,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 청계산입구 초 역세권 신축 오피...
188,34498546,37.447738,127.056370,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 서초구 신원동,청계산입구 초 역세권 오피스텔 사업자용 *청계산입구 초 역세권 신축 오피스텔\r\n...,https://ic.zigbang.com/ic/items/34498546/1.jpg,월세,1000,88.0,10.0,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 사업자용 청계산입구 초 역세권 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39941,34512135,37.680000,127.042272,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/345...,서울시 도봉구 도봉동,✔신축급풀옵션_방큰원룸✔역세권_도봉역 4분✔즉시입주 1. 오픈형원룸/수려한 도봉산경...,https://ic.zigbang.com/ic/items/34512135/1.jpg,월세,500,37.0,3.0,서울시 도봉구 도봉동 신축급풀옵션 방큰원룸 역세권 도봉역 분 즉시입주 오픈형원룸 수...
39942,34522766,37.680244,127.044085,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/345...,서울시 도봉구 도봉동,✔풀옵션_깨끗✔역세권_1호선도봉역 4분✔버스전용 20개노선 1. 교통편리 : 역세권...,https://ic.zigbang.com/ic/items/34522766/1.jpg,월세,300,30.0,3.0,서울시 도봉구 도봉동 풀옵션 깨끗 역세권 호선도봉역 분 버스전용 개노선 교통편리 역...
39943,34411514,37.680953,127.043451,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 도봉구 도봉동,🌈도봉역 2019년식 투룸 안심대출가능 주차가능🌈 📌매물정보\r\n 도봉역5...,https://ic.zigbang.com/ic/items/34411514/1.jpg,전세,20400,NaN,4.0,서울시 도봉구 도봉동 도봉역 년식 투룸 안심대출가능 주차가능 매물정보 도봉역 분 분...
39944,34575820,37.680461,127.043434,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/345...,서울시 도봉구 도봉동,💰HUG 안심전세💰초역세권💰깔끔한 투룸 전세💰 🌈 매물 소개 🌈\r\n\r\n1️⃣...,https://ic.zigbang.com/ic/items/34575820/1.jpg,전세,20000,NaN,4.0,서울시 도봉구 도봉동 안심전세 초역세권 깔끔한 투룸 전세 매물 소개 안심전세 대출 ...


In [5]:
df['doc'][4082]

'서울시 관악구 신림동 신축 화이트톤 신림역 분 깔끔 빌트인 가구 보증금 조정 가능 보증금과 월세 조정 가능 위치 및 교통 신림역 도보 분 이내 버스정류장 도보 분 보안 건물 내부 외부 작동 현관 방 전자도어락 설치 엘리베이터있는 층이며 실제로 보면 더 예쁜 신축 원룸입니다 '

### 직방 크롤링 data'detail'컬럼에서 영어->한글변경

In [6]:
df['doc'] = df['doc'].str.replace('x','불가능')
df['doc'] = df['doc'].str.replace('OK','가능')
df['doc'] = df['doc'].str.replace('O','가능')
df['doc'] = df['doc'].str.replace('X','불가능')
df['doc'] = df['doc'].str.replace('o','가능')
df['doc'] = df['doc'].str.replace('NO','없음')
df['doc'] = df['doc'].str.replace('no','없음')

In [7]:
df['doc'][7005]

'서울시 송파구 삼전동 신축 거실있는 원룸 석촌고분역 도보로 분 직접 찍어온 실매물 위치정보 호선 석촌고분역 도보 분 버스정류장도 가까워 출퇴근하기 좋아요 가격정보 허위 가격아닌 실가격 인근 신축 거실있는 원룸 중 가장 메리트 있는 가격입니다 옵션 에어컨 세탁기 냉장고 인덕션 등 옵션에 포함됩니다 구조나 채광은 어떤가요 효율적으로 공간이용이 가능합니다 분리형 원룸이기 때문에 부엌과 화장실이 방과 나뉘어져 있어요 입주는 언제 가능한가요 신축 첫입주로 언제든 입주 가능하십니다 주변 환경 편의 문화 시설 여부 잠실역과 가까워 쇼핑 맛집 카페 등 문화생활 하기에 골목에 상권이 잘 형성되어 있어 편의시설 석촌호수가 가까이 있어 산책하기 좋아요 주차가능 한가요 주차 대 가능 선착순 만원 이집의 총평 석촌고분역 도보 분 역세권이고 버스 정류장도 가까이에 위치하여 정말 살기 좋으실거에요 인근 매물중에 이만한 매물은 정말 찾기 힘드실거에요 사진으로 보시는것보다 직접 보시면 더욱 더 맘에 드시는 방입니다 '

### 전처리 : 숫자, 영어 특수문자 삭제

In [8]:
df['doc_new'] = df.doc.str.replace("[^가-힣 ]"," ") # 한글만 남기고 특수문자, 숫자, 영어 삭제

C:\Users\user\AppData\Local\Temp\ipykernel_26296\849166722.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['doc_new'] = df.doc.str.replace("[^가-힣 ]"," ") # 한글만 남기고 특수문자, 숫자, 영어 삭제


In [9]:
df.doc_new = df.doc_new.str.replace("\s+", " ") # white space 삭제

C:\Users\user\AppData\Local\Temp\ipykernel_26296\785149045.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df.doc_new = df.doc_new.str.replace("\s+", " ") # white space 삭제


In [10]:
df

,id,lat,lng,info,url,address,detail,picture,type,deposit,rental_fee,maintenance_fee,doc,doc_new
144,34402581,37.446504,127.057650,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 서초구 신원동,"◆◆전입가,무융자,전세대출가,지하철역3분◆이중창,주차가능 ◆◆전입가,무융자,전세대출...",https://ic.zigbang.com/ic/items/34402581/1.jpg,전세,17500,NaN,8.0,서울시 서초구 신원동 전입가 무융자 전세대출가 지하철역 분 이중창 주차가능 전입가 ...,서울시 서초구 신원동 전입가 무융자 전세대출가 지하철역 분 이중창 주차가능 전입가 ...
145,34448303,37.446532,127.057644,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 서초구 신원동,청계산입구 초 역세권 오피스텔 *청계산입구 초 역세권 신축 오피스텔\r\n*강남역3...,https://ic.zigbang.com/ic/items/34448303/1.jpg,월세,1000,65.0,10.0,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 청계산입구 초 역세권 신축 오피...,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 청계산입구 초 역세권 신축 오피...
159,34521223,37.447032,127.056570,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/345...,서울시 서초구 신원동,★신축급 풀옵션 투룸 오피스텔 전세 매물★ # 12월8일 이후부터 투어 가능\r\...,https://ic.zigbang.com/ic/items/34521223/1.jpg,전세,37500,NaN,20.0,서울시 서초구 신원동 신축급 풀옵션 투룸 오피스텔 전세 매물 월 일 이후부터 투어 ...,서울시 서초구 신원동 신축급 풀옵션 투룸 오피스텔 전세 매물 월 일 이후부터 투어 ...
165,34493219,37.446730,127.056288,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 서초구 신원동,청계산입구 초 역세권 오피스텔 *청계산입구 초 역세권 신축 오피스텔\r\n*강남역3...,https://ic.zigbang.com/ic/items/34493219/1.jpg,전세,17000,NaN,8.0,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 청계산입구 초 역세권 신축 오피...,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 청계산입구 초 역세권 신축 오피...
188,34498546,37.447738,127.056370,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 서초구 신원동,청계산입구 초 역세권 오피스텔 사업자용 *청계산입구 초 역세권 신축 오피스텔\r\n...,https://ic.zigbang.com/ic/items/34498546/1.jpg,월세,1000,88.0,10.0,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 사업자용 청계산입구 초 역세권 ...,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 사업자용 청계산입구 초 역세권 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39941,34512135,37.680000,127.042272,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/345...,서울시 도봉구 도봉동,✔신축급풀옵션_방큰원룸✔역세권_도봉역 4분✔즉시입주 1. 오픈형원룸/수려한 도봉산경...,https://ic.zigbang.com/ic/items/34512135/1.jpg,월세,500,37.0,3.0,서울시 도봉구 도봉동 신축급풀옵션 방큰원룸 역세권 도봉역 분 즉시입주 오픈형원룸 수...,서울시 도봉구 도봉동 신축급풀옵션 방큰원룸 역세권 도봉역 분 즉시입주 오픈형원룸 수...
39942,34522766,37.680244,127.044085,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/345...,서울시 도봉구 도봉동,✔풀옵션_깨끗✔역세권_1호선도봉역 4분✔버스전용 20개노선 1. 교통편리 : 역세권...,https://ic.zigbang.com/ic/items/34522766/1.jpg,월세,300,30.0,3.0,서울시 도봉구 도봉동 풀옵션 깨끗 역세권 호선도봉역 분 버스전용 개노선 교통편리 역...,서울시 도봉구 도봉동 풀옵션 깨끗 역세권 호선도봉역 분 버스전용 개노선 교통편리 역...
39943,34411514,37.680953,127.043451,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 도봉구 도봉동,🌈도봉역 2019년식 투룸 안심대출가능 주차가능🌈 📌매물정보\r\n 도봉역5...,https://ic.zigbang.com/ic/items/34411514/1.jpg,전세,20400,NaN,4.0,서울시 도봉구 도봉동 도봉역 년식 투룸 안심대출가능 주차가능 매물정보 도봉역 분 분...,서울시 도봉구 도봉동 도봉역 년식 투룸 안심대출가능 주차가능 매물정보 도봉역 분 분...
39944,34575820,37.680461,127.043434,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/345...,서울시 도봉구 도봉동,💰HUG 안심전세💰초역세권💰깔끔한 투룸 전세💰 🌈 매물 소개 🌈\r\n\r\n1️⃣...,https://ic.zigbang.com/ic/items/34575820/1.jpg,전세,20000,NaN,4.0,서울시 도봉구 도봉동 안심전세 초역세권 깔끔한 투룸 전세 매물 소개 안심전세 대출 ...,서울시 도봉구 도봉동 안심전세 초역세권 깔끔한 투룸 전세 매물 소개 안심전세 대출 ...


In [11]:
df['doc_new'][3009]

'서울시 관악구 봉천동 허위매물 서울대입구역복층 초역세권 위치 및 교통 강남권으로 출퇴근 하시는분들과 주변 대학교로 가는 교통편이 많습니다 대로변 인근으로 버스타시기도 용이합니다 서울대 중앙대 숭실대 학생들에게도 좋은위치에 있어 통학이 편리합니다 주변에 어린이집 유치원과 초 중 고등학교가 있어 아이키우기 좋은위치에 있습니다 주변환경 주변에 은행 시장 편의점 약국 병원 대형마트 세탁소 셀프빨래방등 편의시설이 있습니다 대로변과 가까운 역세권입니다 호선과 버스로 어디든 이동하시기 불편함이없습니다 대로변이라 치안에 안전하고 근처에 학교와 가 많아서 여성분들에게 안전한 위치에 있습니다 '

### 'new_doc'에서 불가능/가능 통일하기

In [12]:
okay=['함께 거주 가능한 집입니다','돼요','입주가능','가능할것같습니다','가능합니다','가능하며']
sorry=['키우는것은 안됨니다','안됨','불가합니다','키우지 못합니다','안되요','사육금지','안됩니다','좀 어려워요','사절합니다',
       '안되며','자리는 없습니다','알아서 하셔야 합니다','사육 안 됩니다','안됩니다','절대 안된다고 하세요','불가','불가능해요',
      '불가해요','안 되십니다','금지','좀 어려워요','어렵습니다','안돼요','불가입니다','알아서 하셔야 합니다','반려','어려운점']
    
for i in tqdm(okay):
    df['doc_new'] = df['doc_new'].str.replace(i,'가능')

for i in tqdm(sorry):
    df['doc_new'] = df['doc_new'].str.replace(i,'불가능')    


100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [00:00<00:00, 37.05it/s]


In [13]:
# sorry=['키우는것은 안됨니다','안됨','불가합니다','키우지 못합니다','안되요','사육금지','안됩니다','좀 어려워요','사절합니다',
#        '안되며','자리는 없습니다','알아서 하셔야 합니다','사육 안 됩니다','안됩니다','절대 안된다고 하세요','불가','불가능해요',
#       '불가해요','안 되십니다','금지','좀 어려워요','어렵습니다','안돼요','불가입니다','알아서 하셔야 합니다','반려']

# new = []
# for j in tqdm(range(len(df['doc_new']))):
#     for i in (sorry):
#         df['doc_new'][j] = df['doc_new'][j].replace(i,'불가')
#     new.append(df['doc_new'][j])
    
             
# df['doc_new']=new

In [15]:
df

,id,lat,lng,info,url,address,detail,picture,type,deposit,rental_fee,maintenance_fee,doc,doc_new
144,34402581,37.446504,127.057650,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 서초구 신원동,"◆◆전입가,무융자,전세대출가,지하철역3분◆이중창,주차가능 ◆◆전입가,무융자,전세대출...",https://ic.zigbang.com/ic/items/34402581/1.jpg,전세,17500,NaN,8.0,서울시 서초구 신원동 전입가 무융자 전세대출가 지하철역 분 이중창 주차가능 전입가 ...,서울시 서초구 신원동 전입가 무융자 전세대출가 지하철역 분 이중창 주차가능 전입가 ...
145,34448303,37.446532,127.057644,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 서초구 신원동,청계산입구 초 역세권 오피스텔 *청계산입구 초 역세권 신축 오피스텔\r\n*강남역3...,https://ic.zigbang.com/ic/items/34448303/1.jpg,월세,1000,65.0,10.0,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 청계산입구 초 역세권 신축 오피...,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 청계산입구 초 역세권 신축 오피...
159,34521223,37.447032,127.056570,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/345...,서울시 서초구 신원동,★신축급 풀옵션 투룸 오피스텔 전세 매물★ # 12월8일 이후부터 투어 가능\r\...,https://ic.zigbang.com/ic/items/34521223/1.jpg,전세,37500,NaN,20.0,서울시 서초구 신원동 신축급 풀옵션 투룸 오피스텔 전세 매물 월 일 이후부터 투어 ...,서울시 서초구 신원동 신축급 풀옵션 투룸 오피스텔 전세 매물 월 일 이후부터 투어 ...
165,34493219,37.446730,127.056288,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 서초구 신원동,청계산입구 초 역세권 오피스텔 *청계산입구 초 역세권 신축 오피스텔\r\n*강남역3...,https://ic.zigbang.com/ic/items/34493219/1.jpg,전세,17000,NaN,8.0,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 청계산입구 초 역세권 신축 오피...,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 청계산입구 초 역세권 신축 오피...
188,34498546,37.447738,127.056370,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 서초구 신원동,청계산입구 초 역세권 오피스텔 사업자용 *청계산입구 초 역세권 신축 오피스텔\r\n...,https://ic.zigbang.com/ic/items/34498546/1.jpg,월세,1000,88.0,10.0,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 사업자용 청계산입구 초 역세권 ...,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 사업자용 청계산입구 초 역세권 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39941,34512135,37.680000,127.042272,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/345...,서울시 도봉구 도봉동,✔신축급풀옵션_방큰원룸✔역세권_도봉역 4분✔즉시입주 1. 오픈형원룸/수려한 도봉산경...,https://ic.zigbang.com/ic/items/34512135/1.jpg,월세,500,37.0,3.0,서울시 도봉구 도봉동 신축급풀옵션 방큰원룸 역세권 도봉역 분 즉시입주 오픈형원룸 수...,서울시 도봉구 도봉동 신축급풀옵션 방큰원룸 역세권 도봉역 분 즉시입주 오픈형원룸 수...
39942,34522766,37.680244,127.044085,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/345...,서울시 도봉구 도봉동,✔풀옵션_깨끗✔역세권_1호선도봉역 4분✔버스전용 20개노선 1. 교통편리 : 역세권...,https://ic.zigbang.com/ic/items/34522766/1.jpg,월세,300,30.0,3.0,서울시 도봉구 도봉동 풀옵션 깨끗 역세권 호선도봉역 분 버스전용 개노선 교통편리 역...,서울시 도봉구 도봉동 풀옵션 깨끗 역세권 호선도봉역 분 버스전용 개노선 교통편리 역...
39943,34411514,37.680953,127.043451,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 도봉구 도봉동,🌈도봉역 2019년식 투룸 안심대출가능 주차가능🌈 📌매물정보\r\n 도봉역5...,https://ic.zigbang.com/ic/items/34411514/1.jpg,전세,20400,NaN,4.0,서울시 도봉구 도봉동 도봉역 년식 투룸 안심대출가능 주차가능 매물정보 도봉역 분 분...,서울시 도봉구 도봉동 도봉역 년식 투룸 안심대출가능 주차가능 매물정보 도봉역 분 분...
39944,34575820,37.680461,127.043434,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/345...,서울시 도봉구 도봉동,💰HUG 안심전세💰초역세권💰깔끔한 투룸 전세💰 🌈 매물 소개 🌈\r\n\r\n1️⃣...,https://ic.zigbang.com/ic/items/34575820/1.jpg,전세,20000,NaN,4.0,서울시 도봉구 도봉동 안심전세 초역세권 깔끔한 투룸 전세 매물 소개 안심전세 대출 ...,서울시 도봉구 도봉동 안심전세 초역세권 깔끔한 투룸 전세 매물 소개 안심전세 대출 ...


In [16]:
df['doc_new'][3009]

'서울시 관악구 봉천동 허위매물 서울대입구역복층 초역세권 위치 및 교통 강남권으로 출퇴근 하시는분들과 주변 대학교로 가는 교통편이 많습니다 대로변 인근으로 버스타시기도 용이합니다 서울대 중앙대 숭실대 학생들에게도 좋은위치에 있어 통학이 편리합니다 주변에 어린이집 유치원과 초 중 고등학교가 있어 아이키우기 좋은위치에 있습니다 주변환경 주변에 은행 시장 편의점 약국 병원 대형마트 세탁소 셀프빨래방등 편의시설이 있습니다 대로변과 가까운 역세권입니다 호선과 버스로 어디든 이동하시기 불편함이없습니다 대로변이라 치안에 안전하고 근처에 학교와 가 많아서 여성분들에게 안전한 위치에 있습니다 '

In [17]:
df['doc_new'] = df['doc_new'].str.replace('불가능능','불가능')

### 불용어사전 + OKT 형태소 -> Token분리

In [19]:
okt = Okt()

In [20]:
#불용어
stopwords = pd.read_csv('ko-stopwords.csv') #한국어불용어사전 불러오기
stopwords=list(stopwords['stopwords']) 
stopwords.extend(['분','대','만원','년의','년','호선','층','년전','룸을','분거리','블럭','년','평의','평','년식','초','층높이',
                  '개월','평대','개','번출구','년차','부터','일','이후','중개사','면적','매물','공인','금액','부동산','사진',
                  '비','사무소','정보','설명','대표','등록','상세','내용','중개','주소','직방','층수','드리다','비용','공급',
                  '요약','제목','직방에','매물입니다','분거리','크기','번','에','등록된','세대','룸','분거리에','층건물의','층집이며',
                  '호선','가','실매물','실거래','일이내','분내','실제','약','크기의','배','현','총','대','참고해주세요','직접',
                  '단독광고매물로','광고','실금액으로','실금액','문의바랍니다','매물정보','실면적','평으로','월','만원','만원입니다',
                  '촬영한','실사진','급','는','가','편 하 게 연 락 주 시 면 친 절 하 게 상 담 하 여 성 실 하 게 딱 맞 는 집 으 로 찾 아 드 리 겠 습 니 다',
                  '거리','을','개월','개월은','억','천만원','일이후','은','실','제','청사','등등','인','가','화','미터','이내','종','류',
                  ' 에 직방에 등록된 매물입니다 ','에서','고','이다','는','한','씨', "것","거","게","데","이다","건","고","되다","되어다","걸","기",
                  "시","네","듯","랍니","중이","얘","스","도도", "나","수","개","내","기","제","저","인","있다","이렇다",
                  "그렇다","번","위","팅","분","인","링","란","포","두", "진짜", "하다" ,"이다" ,"가다", "이제" ,"들다"
                 "더", "안", '때문에'
                 ]) #불용어 추가

stopwords=set(stopwords)    

# 형태소 분석 함수 만들기
def okt_pos_tagging(string):
    pos_words = okt.pos(string, stem=True, norm=True) # 형태소 분석. 단어는 사전형으로 바꿔주기
    words = [word for word, tag in pos_words if tag in ['Noun', 'Adjective', 'Verb','Adverb'] if word not in stopwords ]
    return words

In [21]:
# 형태소 분석 적용
tokenizing_doc=[] 
for i in tqdm(df['doc_new']):
    tokenizing_doc.append(okt_pos_tagging(i))
df['token_new'] = tokenizing_doc #새로운 tokenizing이라는 컬럼에 키워드 형태로 만든 원문을 매칭

100%|████████████████████████████████████████████████████████████████████████████| 30637/30637 [10:47<00:00, 47.32it/s]


In [22]:
df

,id,lat,lng,info,url,address,detail,picture,type,deposit,rental_fee,maintenance_fee,doc,doc_new,token_new
144,34402581,37.446504,127.057650,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 서초구 신원동,"◆◆전입가,무융자,전세대출가,지하철역3분◆이중창,주차가능 ◆◆전입가,무융자,전세대출...",https://ic.zigbang.com/ic/items/34402581/1.jpg,전세,17500,NaN,8.0,서울시 서초구 신원동 전입가 무융자 전세대출가 지하철역 분 이중창 주차가능 전입가 ...,서울시 서초구 신원동 전입가 무융자 전세대출가 지하철역 분 이중창 주차가능 전입가 ...,"[서울시, 서초구, 신원동, 전입, 무, 융자, 대출, 지하철역, 이중창, 주차, ..."
145,34448303,37.446532,127.057644,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 서초구 신원동,청계산입구 초 역세권 오피스텔 *청계산입구 초 역세권 신축 오피스텔\r\n*강남역3...,https://ic.zigbang.com/ic/items/34448303/1.jpg,월세,1000,65.0,10.0,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 청계산입구 초 역세권 신축 오피...,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 청계산입구 초 역세권 신축 오피...,"[서울시, 서초구, 신원동, 청계산, 입구, 역다, 권, 오피스텔, 청계산, 입구,..."
159,34521223,37.447032,127.056570,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/345...,서울시 서초구 신원동,★신축급 풀옵션 투룸 오피스텔 전세 매물★ # 12월8일 이후부터 투어 가능\r\...,https://ic.zigbang.com/ic/items/34521223/1.jpg,전세,37500,NaN,20.0,서울시 서초구 신원동 신축급 풀옵션 투룸 오피스텔 전세 매물 월 일 이후부터 투어 ...,서울시 서초구 신원동 신축급 풀옵션 투룸 오피스텔 전세 매물 월 일 이후부터 투어 ...,"[서울시, 서초구, 신원동, 신축, 풀, 옵션, 투룸, 오피스텔, 전세, 투어, 가..."
165,34493219,37.446730,127.056288,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 서초구 신원동,청계산입구 초 역세권 오피스텔 *청계산입구 초 역세권 신축 오피스텔\r\n*강남역3...,https://ic.zigbang.com/ic/items/34493219/1.jpg,전세,17000,NaN,8.0,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 청계산입구 초 역세권 신축 오피...,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 청계산입구 초 역세권 신축 오피...,"[서울시, 서초구, 신원동, 청계산, 입구, 역다, 권, 오피스텔, 청계산, 입구,..."
188,34498546,37.447738,127.056370,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 서초구 신원동,청계산입구 초 역세권 오피스텔 사업자용 *청계산입구 초 역세권 신축 오피스텔\r\n...,https://ic.zigbang.com/ic/items/34498546/1.jpg,월세,1000,88.0,10.0,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 사업자용 청계산입구 초 역세권 ...,서울시 서초구 신원동 청계산입구 초 역세권 오피스텔 사업자용 청계산입구 초 역세권 ...,"[서울시, 서초구, 신원동, 청계산, 입구, 역다, 권, 오피스텔, 사업자, 용, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39941,34512135,37.680000,127.042272,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/345...,서울시 도봉구 도봉동,✔신축급풀옵션_방큰원룸✔역세권_도봉역 4분✔즉시입주 1. 오픈형원룸/수려한 도봉산경...,https://ic.zigbang.com/ic/items/34512135/1.jpg,월세,500,37.0,3.0,서울시 도봉구 도봉동 신축급풀옵션 방큰원룸 역세권 도봉역 분 즉시입주 오픈형원룸 수...,서울시 도봉구 도봉동 신축급풀옵션 방큰원룸 역세권 도봉역 분 즉시입주 오픈형원룸 수...,"[서울시, 도봉구, 도봉동, 신축, 풀, 옵션, 방, 크다, 원룸, 역다, 권, 도..."
39942,34522766,37.680244,127.044085,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/345...,서울시 도봉구 도봉동,✔풀옵션_깨끗✔역세권_1호선도봉역 4분✔버스전용 20개노선 1. 교통편리 : 역세권...,https://ic.zigbang.com/ic/items/34522766/1.jpg,월세,300,30.0,3.0,서울시 도봉구 도봉동 풀옵션 깨끗 역세권 호선도봉역 분 버스전용 개노선 교통편리 역...,서울시 도봉구 도봉동 풀옵션 깨끗 역세권 호선도봉역 분 버스전용 개노선 교통편리 역...,"[서울시, 도봉구, 도봉동, 풀, 옵션, 깨끗, 역다, 권, 도봉역, 버스, 전용,..."
39943,34411514,37.680953,127.043451,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/344...,서울시 도봉구 도봉동,🌈도봉역 2019년식 투룸 안심대출가능 주차가능🌈 📌매물정보\r\n 도봉역5...,https://ic.zigbang.com/ic/items/34411514/1.jpg,전세,20400,NaN,4.0,서울시 도봉구 도봉동 도봉역 년식 투룸 안심대출가능 주차가능 매물정보 도봉역 분 분...,서울시 도봉구 도봉동 도봉역 년식 투룸 안심대출가능 주차가능 매물정보 도봉역 분 분...,"[서울시, 도봉구, 도봉동, 도봉역, 투룸, 안심, 대출, 가능, 주차, 가능, 도..."
39944,34575820,37.680461,127.043434,{'url': 'https://www.zigbang.com/home/oneroom/...,https://www.zigbang.com/home/oneroom/items/345...,서울시 도봉구 도봉동,💰HUG 안심전세💰초역세권💰깔끔한 투룸 전세💰 🌈 매물 소개 🌈\r\n\r\n1️⃣...,https://ic.zigbang.com/ic/items/34575820/1.jpg,전세,20000,NaN,4.0,서울시 도봉구 도봉동 안심전세 초역세권 깔끔한 투룸 전세 매물 소개 안심전세 대출 ...,서울시 도봉구 도봉동 안심전세 초역세권 깔끔한 투룸 전세 매물 소개 안심전세 대출 ...,"[서울시, 도봉구, 도봉동, 안심, 전세, 역다, 권, 깔끔하다, 투룸, 전세, 소..."


In [24]:
df['token_new'][20000]

['서울시',
 '성북구',
 '삼선동',
 '풀',
 '옵션',
 '성북천',
 '변',
 '채광',
 '좋다',
 '럭셔리',
 '원룸',
 '소개',
 '성신여대입구역',
 '버스정류장',
 '넓다',
 '방',
 '신축',
 '깔끔하다',
 '컨디션',
 '창',
 '채광',
 '환기',
 '좋다',
 '풀',
 '옵션',
 '생활',
 '편리하다',
 '전입',
 '신고',
 '확정',
 '일자',
 '가능',
 '안전하다',
 '좋다',
 '집다',
 '금방',
 '나가다',
 '서두르다',
 '선점']

In [25]:
df.to_pickle('seoul_room_1220.pkl') 

In [ ]:
df.drop(['index'], axis=1, inplace = True)